In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
base = pd.read_csv("/kaggle/input/moneyball-mlb-stats-19622012/baseball.csv")
base.head()

In [ ]:
base.count()

DePodesta predicted 2002 season strategy using 1962-2001 data.

In [ ]:
dpyears = base[base.Year < 2002]  
dpyears.head()

In [ ]:
dpyears.count() #number of observations decreased because of year range

Number of Teams in MLB!

In [ ]:
# check = base[base.Year == 2002]
# teamcount = check['Team'].count()
# teamcount = base[base.Year == 2001][base.Playoffs == 1]['Team'].count()
teamcount = base[base.Year == 2001]['Team'].count()
teamcount

In [ ]:
teams = base.Team.value_counts()
teams

DePodesta predicted that if can win 95 in a season you are almost certainly qualified for the Playoffs!!  

In [ ]:
qualifiedwins = base[base.Year < 2002]
# qualifiedwins
qualifiedwinsnew = qualifiedwins[['Team','Year','W','Playoffs']]
# qualifiedwinsnew = qualifiedwinsnew[qualifiedwinsnew.Playoffs == 1]
qualifiedwinsnew

In [ ]:
import seaborn as sns; 
import matplotlib.pyplot as plt
# tips = sns.load_dataset("qualifiedwinsnew")
plt.figure(figsize=(10,9))
ax = sns.scatterplot(x="W", y="Team", hue="Playoffs",data=qualifiedwinsnew)
plt.plot(95, 0, color='r')

Now we know you 95 wins in a season to make the playoffs!

But how much more do you score than give away?

DePodesta predicted 135 runs difference to make the playoffs with 95 wins. Let's see!

In [ ]:
runs = base[base.Year < 2002]
runs.info()

In [ ]:
runs['RD'] = runs['RS'] - runs['RA']
runs.info()

In [ ]:
sns.lmplot(x ="RD", y ="W", data = runs, order = 2, ci = None)

Plot shows DePodesta might be right. But lets be certain.

In [ ]:
from sklearn import preprocessing, svm 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 

X = np.array(runs['RD']).reshape(-1, 1) 
y = np.array(runs['W']).reshape(-1, 1) 
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25) 
  
# Splitting the data into training and testing data 
regr = LinearRegression() 
  
m = regr.fit(X_train, y_train)
m.get_params()

In [ ]:
print(regr.score(X_test, y_test))

In [ ]:
regr.predict(np.array(133).reshape(-1, 1))

In [ ]:
import statsmodels.formula.api as smf
smf.ols(formula ='W ~ RD',data=runs).fit().summary()

We have linear regression relationship:

Wins = 80.8814 + 0.1058*RD

So if wins are 95 the run difference is calculated as 

RD = (95 - 80.8814)/0.1058 ~ 133

Close Enough!!!

Now how to get there? 

For this he used 2001 data. He found out that On Base Percentage and Slugging Rate are the most important attributes to predict the runs scored.  

In [ ]:
# runscored = runs[runs.Year == 2001]
smf.ols(formula ='RS ~ OBP + SLG',data=runs).fit().summary()

Equation is
RS = -804.6271 + 2737.7680(OBP) + 1584.9086(SLG)

So if put in 2001 values of Oakland Athletics we get 

In [ ]:
obp = runs[runs.Year == 2001][runs.Team == 'OAK'][['Team','OBP','SLG']]
obp

If we put these values in we get that they will score close to 835 runs.  

Similarly, for Run Allowed. 

In [ ]:
smf.ols(formula ='RA ~ OOBP + OSLG',data=runs).fit().summary()

RA = -837.3779 + 2913.6(OOBP) + 1514.3(OSLG)


In [ ]:
oobp = runs[runs.Year == 2001][runs.Team == 'OAK'][['Team','OOBP','OSLG']]
oobp

635 runs.

Now if we put these values of RS and RA we can predict wins in 2002.

Wins = 80.8814 + 0.1058 * RD

Wins = 102

Which is very close to the actual wins (103)!!!!